# Project Description: Twitter US Airline Sentiment

### Importing libraries, loading the dataset, printing shape of data & data description. (5 marks)

In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

from bs4 import BeautifulSoup
import re, string, unicodedata
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer
# from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

from nltk.tokenize import word_tokenize
#from nltk.stem import PorterStemmer

from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

##### (This section was for personal documentation in order to make TensorFlow-Keras to run in my machine.)
### Dokumentation zur Systemkonfiguration:

Beginnen Sie mit der rechten Maustaste auf Start. (Wählen Sie als Administrator die Command Prompt aus.)
Führen Sie den folgenden Befehl aus:

    C:\ProgramData\Anaconda3\Scripts\activate.bat

(Quelle: https://stackoverflow.com/questions/47914980/how-to-access-anaconda-command-prompt-in-windows-10-64-bit)

Zu textblob und vaderSentimen installieren:

    conda install -c conda-forge textblob
    python -m textblob.download_corpora

    pip install vaderSentiment

Zu nltk installieren:

    import nltk
    nltk.download()
    
(Quelle: https://ilmoirfan.com/installation-of-natural-language-toolkit-nltk-on-anaconda-2/)

    conda install -c anaconda nltk

(Quelle: https://anaconda.org/anaconda/nltk)

Conda update:

    (conda update -n base -c defaults conda)

In [2]:
# nötig
dfTweets = pd.read_csv("Tweets.csv")

In [3]:
dfTweets.columns

Index(['tweet_id', 'airline_sentiment', 'airline_sentiment_confidence',
       'negativereason', 'negativereason_confidence', 'airline',
       'airline_sentiment_gold', 'name', 'negativereason_gold',
       'retweet_count', 'text', 'tweet_coord', 'tweet_created',
       'tweet_location', 'user_timezone'],
      dtype='object')

In [4]:
dfTweets.shape

(14640, 15)

In [5]:
dfTweets.describe()

,tweet_id,airline_sentiment_confidence,negativereason_confidence,retweet_count
count,1.464000e+04,14640.000000,10522.000000,14640.000000
mean,5.692184e+17,0.900169,0.638298,0.082650
std,7.791112e+14,0.162830,0.330440,0.745778
min,5.675883e+17,0.335000,0.000000,0.000000
25%,5.685592e+17,0.692300,0.360600,0.000000
50%,5.694779e+17,1.000000,0.670600,0.000000
75%,5.698905e+17,1.000000,1.000000,0.000000
max,5.703106e+17,1.000000,1.000000,44.000000


### Dropping all columns except "text" and "airline_sentiment", shape of data & printing first 5 rows (5 marks)

In [6]:
# nötig
dfTweets = dfTweets.drop("tweet_id", axis=1)
dfTweets = dfTweets.drop("airline_sentiment_confidence", axis=1)
dfTweets = dfTweets.drop("negativereason", axis=1)
dfTweets = dfTweets.drop("negativereason_confidence", axis=1)
dfTweets = dfTweets.drop("airline", axis=1)
dfTweets = dfTweets.drop("airline_sentiment_gold", axis=1)
dfTweets = dfTweets.drop("name", axis=1)
dfTweets = dfTweets.drop("negativereason_gold", axis=1)
dfTweets = dfTweets.drop("retweet_count", axis=1)
dfTweets = dfTweets.drop("tweet_coord", axis=1)
dfTweets = dfTweets.drop("tweet_created", axis=1)
dfTweets = dfTweets.drop("tweet_location", axis=1)
dfTweets = dfTweets.drop("user_timezone", axis=1)

In [7]:
dfTweets.shape

(14640, 2)

In [8]:
dfTweets.head()

,airline_sentiment,text
0,neutral,@VirginAmerica What @dhepburn said.
1,positive,@VirginAmerica plus you've added commercials t...
2,neutral,@VirginAmerica I didn't today... Must mean I n...
3,negative,@VirginAmerica it's really aggressive to blast...
4,negative,@VirginAmerica and it's a really big bad thing...


In [9]:
dfTweets.info();

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14640 entries, 0 to 14639
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   airline_sentiment  14640 non-null  object
 1   text               14640 non-null  object
dtypes: object(2)
memory usage: 228.9+ KB


In [10]:
for col in dfTweets:
    print(col," = ",dfTweets[col].unique())

airline_sentiment  =  ['neutral' 'positive' 'negative']
text  =  ['@VirginAmerica What @dhepburn said.'
 "@VirginAmerica plus you've added commercials to the experience... tacky."
 "@VirginAmerica I didn't today... Must mean I need to take another trip!"
 ... '@AmericanAir Please bring American Airlines to #BlackBerry10'
 "@AmericanAir you have my money, you change my flight, and don't answer your phones! Any other suggestions so I can make my commitment??"
 '@AmericanAir we have 8 ppl so we need 2 know how many seats are on the next flight. Plz put us on standby for 4 people on the next flight?']


### Text pre-processing: Data preparation. (20 Marks)

#### HTML tag removal.
<h4 style="color:red;">Tokenization. IMPORTANT: This part of the code might be within the different the part of the code regarding vectorization.</h4>

#### Number remotion.
#### Special Character  and Punctuation remotion.
#### Conversion to lowercase.
<h4 style="color:red;">Lemmatization and Stemming.. IMPORTANT: This part of the code might be within the different the part of the code regarding vectorization.</h4>

#### Joining the words back.
<h4 style="color:red;">Printing first 5 rows of data after pre-processing. IMPORTANT: This part of the code might be within the different the part of the code regarding vectorization.</h4>

In [11]:
print (stopwords.words("english")) 

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [12]:
stemmer = LancasterStemmer()
stop_words = set(stopwords.words("english"))

In [13]:
def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []                        # Create empty list to store pre-processed words.
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)        # Append processed words to new list.
    return new_words

def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []                        # Create empty list to store pre-processed words.
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)    # Append processed words to new list.
    return new_words

def stem_words(words):
    """Stem words in list of tokenized words"""
    #stemmer = LancasterStemmer()
    stems = []                            # Create empty list to store pre-processed words.
    for word in words:
        stem = stemmer.stem(word)
        stems.append(stem)                # Append processed words to new list.
    return stems

def lemmatize_verbs(words):
    """Lemmatize verbs in list of tokenized words"""
    lemmatizer = WordNetLemmatizer()
    lemmas = []                           # Create empty list to store pre-processed words.
    for word in words:
        lemma = lemmatizer.lemmatize(word, pos='v')
        lemmas.append(lemma)              # Append processed words to new list.
    return lemmas

#def stem_and_lemmatize(words):
#    stems = stem_words(words)
#    lemmas = lemmatize_verbs(words)
#    return stems, lemmas

In [14]:
# Function to convert a raw review to a string of words
# The input is a single string (a raw movie review), and 
# the output is a single string (a preprocessed movie review)
def review_to_words( raw_review ):
    #
    # Normalization.
    #
    # 1. Removing HTML
    review_text = BeautifulSoup(raw_review).get_text() 
    #
    # 2. Removing non-letters, remove punctuation and numbers
    letters_only = re.sub("[^a-zA-Z]", " ", review_text)
    #
    # 3. Converting to lower case, splitting into individual words
    words = letters_only.lower().split()                             
    #
    # 4. In Python, searching a set is much faster than searching
    #   a list, so convertting the stop words to a set
    #stop_words = set(stopwords.words("english"))                  
    # 
    # 5. Removing stop words
    meaningful_words = [w for w in words if not w in stop_words]
    #
    # 6. Removing non ascii characters
    meaningful_words = remove_non_ascii(meaningful_words)
    #
    # 7. Removing non English punctuation
    meaningful_words = remove_punctuation(meaningful_words)
    #
    meaningful_words_lemmatized = lemmatize_verbs(meaningful_words)
    #
    # 8. Joining the words back into one string separated by space, 
    # and returning the result.
    return( " ".join( meaningful_words_lemmatized ))  

In [15]:
# Get the number of Tweets based on the dataframe column size
numTweets = dfTweets["text"].size

# Initializing an empty list to hold the clean Tweets
cleanTweets = []

# Loop over each Tweet; create an index i that goes from 0 to the length
# of the Tweets list 
for i in range( 0, numTweets ):
    # Call our function for each Tweet, and then add the result to the list of
    # clean Tweets
    cleanTweets.append( review_to_words( dfTweets["text"][i] ) )

In [16]:
cleanTweets

['virginamerica dhepburn say',
 'virginamerica plus add commercials experience tacky',
 'virginamerica today must mean need take another trip',
 'virginamerica really aggressive blast obnoxious entertainment guests face little recourse',
 'virginamerica really big bad thing',
 'virginamerica seriously would pay flight seat play really bad thing fly va',
 'virginamerica yes nearly every time fly vx ear worm go away',
 'virginamerica really miss prime opportunity men without hat parody https co mwpg grezp',
 'virginamerica well',
 'virginamerica amaze arrive hour early good',
 'virginamerica know suicide second lead cause death among teens',
 'virginamerica pretty graphics much better minimal iconography',
 'virginamerica great deal already think nd trip australia even go st trip yet p',
 'virginamerica virginmedia fly fabulous seductive sky u take stress away travel http co ahlxhhkiyn',
 'virginamerica thank',
 'virginamerica sfo pdx schedule still mia',
 'virginamerica excite first cro

### Vectorization (10 marks) (first part)

#### Using CountVectorizer

In [17]:
print ("Creating the bag of words...\n")
#from sklearn.feature_extraction.text import CountVectorizer

# Initialize the "CountVectorizer" object, which is scikit-learn's bag of words tool.  
vectorizer = CountVectorizer(analyzer = "word", tokenizer = None, preprocessor = None, stop_words = None, max_features = 5000) 

# fit_transform() does two functions: First, it fits the model
# and learns the vocabulary; second, it transforms our training data
# into feature vectors. The input to fit_transform should be a list of 
# strings.
dataFeaturesTweets = vectorizer.fit_transform(cleanTweets)

# Numpy arrays are easy to work with, so convert the result to an 
# array
dataFeaturesTweets = dataFeaturesTweets.toarray()

Creating the bag of words...



In [18]:
dataFeaturesTweets.shape

(14640, 5000)

In [19]:
dataFeaturesTweets

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [20]:
# Take a look at the words in the vocabulary
vocabulary = vectorizer.get_feature_names()
print (vocabulary)

['aa', 'aadv', 'aadvantage', 'aafail', 'aal', 'aarp', 'abandon', 'abc', 'abcnetwork', 'abi', 'abilities', 'ability', 'able', 'aboard', 'abq', 'abroad', 'absolute', 'absolutely', 'absurd', 'abt', 'abuse', 'abysmal', 'ac', 'accept', 'acceptable', 'access', 'accident', 'accidentally', 'accommodate', 'accommodation', 'accommodations', 'accomplish', 'accord', 'accordingly', 'account', 'accountability', 'accrue', 'acct', 'accts', 'accurate', 'accuse', 'acebo', 'achieve', 'acknowledge', 'acknowledgement', 'acknowledgment', 'acquire', 'acquisition', 'across', 'act', 'action', 'activate', 'active', 'activities', 'actual', 'actually', 'ad', 'adam', 'add', 'addition', 'additional', 'address', 'addtl', 'adjacent', 'admiral', 'admirals', 'admit', 'adopt', 'adore', 'ads', 'adult', 'advance', 'advantage', 'advertise', 'advice', 'advise', 'advisories', 'advisory', 'aerojobmarket', 'affect', 'affiliate', 'afford', 'affordable', 'afraid', 'afterall', 'afternoon', 'age', 'agency', 'agent', 'agents', 'agg

In [21]:
# Sum up the counts of each vocabulary word
dist = np.sum(dataFeaturesTweets, axis=0)

# For each, print the vocabulary word and the number of times it appears in the training set
for tag, count in zip(vocabulary, dist):
    print (count, tag)

295 aa
2 aadv
10 aadvantage
2 aafail
2 aal
3 aarp
3 abandon
11 abc
3 abcnetwork
2 abi
2 abilities
5 ability
121 able
4 aboard
9 abq
2 abroad
22 absolute
30 absolutely
13 absurd
7 abt
3 abuse
6 abysmal
5 ac
32 accept
23 acceptable
39 access
5 accident
4 accidentally
26 accommodate
2 accommodation
8 accommodations
4 accomplish
11 accord
4 accordingly
82 account
6 accountability
2 accrue
11 acct
2 accts
10 accurate
2 accuse
2 acebo
3 achieve
4 acknowledge
2 acknowledgement
2 acknowledgment
2 acquire
2 acquisition
16 across
18 act
13 action
2 activate
5 active
3 activities
21 actual
86 actually
5 ad
2 adam
128 add
4 addition
17 additional
55 address
3 addtl
2 adjacent
5 admiral
12 admirals
6 admit
6 adopt
2 adore
5 ads
2 adult
14 advance
15 advantage
11 advertise
9 advice
20 advise
2 advisories
25 advisory
2 aerojobmarket
15 affect
3 affiliate
13 afford
3 affordable
2 afraid
2 afterall
30 afternoon
15 age
6 agency
265 agent
156 agents
3 aggravate
3 aggressive
2 agnt
134 ago
9 agree
3 agt
1

2 compound
2 comps
46 computer
14 computers
4 con
2 concentrate
3 concept
44 concern
4 concert
6 concourse
2 condescend
20 condition
2 condom
2 conection
27 conf
10 conference
3 confidence
4 confident
76 confirm
70 confirmation
2 confirmations
6 conflict
15 confuse
7 confusion
17 congrats
4 congratulations
132 connect
153 connection
36 connections
4 connector
4 consecutive
33 consider
2 considerable
5 consideration
2 consistency
9 consistent
8 consistently
2 consolation
6 constant
5 constantly
2 construction
2 consumer
3 consumers
9 cont
122 contact
3 contain
2 contd
7 content
12 contest
2 context
6 continental
3 contingency
4 continually
41 continue
4 continuous
6 contract
32 control
2 controllers
3 convenient
2 conveniently
5 conversation
3 conversations
2 conveyer
4 convince
3 convo
4 cookies
64 cool
3 coordinate
2 copilot
8 copy
2 corevalues
2 corner
5 corp
16 corporate
7 corporation
45 correct
8 correctly
2 cos
80 cost
6 costa
9 costumer
2 costumers
4 cot
2 cots
2 cough
254 could


4 idk
2 iflyswa
2 ignorant
21 ignore
2 ih
2 ilc
10 ill
5 illegal
58 im
20 imagine
46 imaginedragons
2 imma
3 immediate
10 immediately
3 imo
7 impact
2 impend
19 important
19 impossible
24 impress
3 impression
2 impressions
6 impressive
19 improve
3 improvement
3 improvements
3 inability
3 inaccurate
5 inadequate
5 inappropriate
12 inbound
3 inbox
2 inc
6 incentive
8 inch
9 incident
2 incl
5 inclement
29 include
5 incoming
17 incompetence
17 incompetent
2 inconsiderate
2 inconsistent
36 inconvenience
11 inconvenient
2 inconvience
4 incorrect
4 incorrectly
9 increase
11 incredible
16 incredibly
6 incur
10 ind
7 indeed
2 india
4 indianapolis
8 indicate
3 indication
3 individual
9 industry
5 indy
2 inefficiency
4 inefficient
3 inept
3 inevitable
7 inexcusable
2 inexperienced
16 infant
2 infants
3 inferior
21 inflight
147 info
21 inform
54 information
2 informative
2 infuriate
7 ing
9 initial
2 injure
5 injury
4 innovation
2 inop
3 input
5 inquire
5 ins
13 insane
2 insanity
18 inside
2 insi

4 outrageous
18 outside
8 outsource
13 outstanding
2 outta
6 overall
4 overbook
17 overbooked
4 overcharge
3 overflight
3 overflow
2 overhaul
29 overhead
2 overhear
2 overheat
2 overload
30 overnight
2 overprice
3 overseas
2 overselling
3 oversize
2 oversized
4 oversold
6 overweight
4 overwhelm
3 ow
8 owe
4 owner
2 oy
4 oz
11 pa
5 pacific
10 pack
6 package
2 pafe
32 page
5 pain
7 painful
2 painfully
3 painless
5 paint
13 pair
14 palm
2 panamerican
2 pandora
3 panic
4 pant
14 paper
8 paperwork
3 par
2 parachuteguy
13 parent
12 paris
16 park
3 parker
38 part
4 partial
2 participate
2 particular
5 particularly
2 partly
20 partner
5 partnership
2 partnerships
23 party
3 paso
128 pass
27 passbook
45 passenger
188 passengers
2 passion
2 passive
8 passport
8 password
70 past
5 paste
5 pat
4 patch
3 path
21 pathetic
17 patience
17 patient
4 patiently
3 patrick
2 patronize
3 pattern
2 pattonoswalt
2 paulgordonbrown
3 paulo
12 pax
283 pay
10 payment
5 paypal
17 pbi
2 pbpinftworth
2 pd
19 pdx
2 p

8 spiritairlines
2 spite
10 split
4 spoil
3 sport
11 spot
2 spotify
2 spotty
7 spread
16 spring
2 sprinkle
3 sprint
2 square
2 squawk
3 srq
3 srsly
4 srvc
124 st
195 staff
2 staffer
2 stairs
3 stall
37 stand
8 standard
4 standards
51 standby
2 standbye
2 standbys
14 star
8 staralliance
2 starbucks
128 start
2 startle
3 starve
6 stat
35 state
9 statement
5 station
114 status
75 stay
20 steal
2 steam
5 steamboat
2 steel
2 steer
6 stellar
31 step
2 stephenrodrick
2 steve
15 stewardess
3 stewardesses
2 sti
187 stick
2 sticker
586 still
2 stillnobags
3 stillwaiting
8 stink
15 stl
2 stndby
5 stock
2 stocker
2 stoke
3 stone
117 stop
3 storage
2 store
4 stories
40 storm
24 story
3 stow
16 straight
2 straighten
97 strand
4 strange
2 stranger
4 strangers
4 strategy
3 stray
6 stream
21 street
13 stress
8 stressful
2 stretch
8 strike
3 strip
2 strive
8 stroller
2 strollers
5 strong
3 strongly
2 structure
2 struggle
5 stt
2 stuckinmemphis
8 student
4 students
25 stuff
5 stun
8 stupid
6 style
3 sub


### Fitting and evaluating the model using both types of vectorization (6+6 Marks) (first part)

#### Fitting and evaluating the model having used CountVectorizer

In [22]:
# Initialize a Random Forest classifier with 100 trees
forest = RandomForestClassifier(verbose=2,n_jobs=-1,n_estimators = 100) 
# Fit the forest to the training set, using the bag of words as 
# features and the sentiment labels as the response variable
#
# This may take a few minutes to run
print ("Training the random forest...")
forest = forest.fit( dataFeaturesTweets, dfTweets["airline_sentiment"] )

# random forest performance through cross vaidation 
print (forest)
print (np.mean(cross_val_score(forest, dataFeaturesTweets, dfTweets["airline_sentiment"],cv=10)))

Training the random forest...


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.


building tree 1 of 100building tree 2 of 100building tree 3 of 100building tree 4 of 100



building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100


[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  1.6min


building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100
building tree 45 of 100
building tree 46 of 100
building tree 47 of 100
building tree 48 of 100
building tree 49 of 100
building tree 50 of 100
building tree 51 of 100
building tree 52 of 100
building tree 53 of 100
building tree 54 of 100
building tree 55 of 100
building tree 56 of 100
building tree 57 of 100
building tree 58 of 100
building tree 59 of 100
building tree 60 of 100
building tree 61 of 100
building tree 62 of 100
building tree 63 of 100
building tree 64 of 100
building tree 65 of 100
building tree 66 of 100
building tree 67 of 100
building tree 68 of 100
building tree 69 of 100
building tree 70 of 100
building tree 71 of 100
building tree 72 of 100
building tree 73 of 100
building tree 74 of 100
building tree 75 of 100
building tree 76 of 100
building tree 77 of 100
building tree 78

[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  4.7min finished


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=False, random_state=None, verbose=2,
                       warm_start=False)


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  5.3min finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:    3.5s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    4.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  4.9min finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.2s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]:

0.7237021857923498


#### Fitting and evaluating the model having used CountVectorizer in combination with TfidfTransformer

In [23]:
def tokenize(sentence):

    # tokenize text
    tokens = word_tokenize(sentence)
    
    # lemmatize and remove stop words
    tokens = [stemmer.stem(word) for word in tokens if word not in stop_words]

    return tokens

In [24]:
vect = CountVectorizer(tokenizer=tokenize)

In [25]:
dataFeaturesTweets2 = vect.fit_transform(cleanTweets)
transformer = TfidfTransformer()
dataFeaturesTweets2tfidf = transformer.fit_transform(dataFeaturesTweets2)

In [26]:
dataFeaturesTweets2tfidf.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [27]:
dataFeaturesTweets2tfidf.shape

(14640, 9817)

In [28]:
# Initialize a Random Forest classifier with 100 trees
forest = RandomForestClassifier(verbose=2,n_jobs=-1,n_estimators = 100) 
# Fit the forest to the training set, using the bag of words as 
# features and the sentiment labels as the response variable
#
# This may take a few minutes to run
print ("Training the random forest...")
forest = forest.fit( dataFeaturesTweets2tfidf, dfTweets["airline_sentiment"] )
# random forest performance through cross vaidation

# random forest performance through cross vaidation 
print (forest)
print (np.mean(cross_val_score(forest, dataFeaturesTweets2tfidf, dfTweets["airline_sentiment"],cv=10)))

Training the random forest...


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.


building tree 2 of 100building tree 1 of 100building tree 3 of 100building tree 4 of 100



building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100


[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    7.8s


building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100
building tree 45 of 100
building tree 46 of 100
building tree 47 of 100
building tree 48 of 100
building tree 49 of 100
building tree 50 of 100
building tree 51 of 100
building tree 52 of 100
building tree 53 of 100
building tree 54 of 100
building tree 55 of 100
building tree 56 of 100
building tree 57 of 100
building tree 58 of 100
building tree 59 of 100
building tree 60 of 100
building tree 61 of 100
building tree 62 of 100
building tree 63 of 100
building tree 64 of 100
building tree 65 of 100
building tree 66 of 100
building tree 67 of 100
building tree 68 of 100
building tree 69 of 100
building tree 70 of 100
building tree 71 of 100
building tree 72 of 100
building tree 73 of 100
building tree 74 of 100
building tree 75 of 100
building tree 76 of 100
building tree 77 of 100
building tree 78

[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   18.5s finished


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=False, random_state=None, verbose=2,
                       warm_start=False)


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    8.8s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   26.9s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    8.9s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   25.3s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    9.4s
[Parallel(n_jobs=-1)]:

0.7242486338797814


[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   29.1s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished


### Vectorization (10 marks) (second part)

In [29]:
myvocabulary = vocabulary
#myvocabulary

In [30]:
corpusSetTweets = set(cleanTweets)

#### Using TfidfVectorizer

In [31]:
# initialize tf-idf vectorizer object
vectorizer2meth = TfidfVectorizer(vocabulary = myvocabulary, ngram_range = (1,2))

In [32]:
# compute bag of word counts and tf-idf values
dataFeaturesTweets3tfidf = vectorizer2meth.fit_transform(cleanTweets)

In [33]:
# convert sparse matrix to numpy array to view
dataFeaturesTweets3tfidf.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [34]:
dataFeaturesTweets3tfidf.shape

(14640, 5000)

### Fitting and evaluating the model using both types of vectorization (6+6 Marks) (second part)

#### Fitting and evaluating the model having used TfidfVectorizer

In [35]:
# Initialize a Random Forest classifier with 100 trees
forest = RandomForestClassifier(verbose=2,n_jobs=-1,n_estimators = 100) 
# Fit the forest to the training set, using the bag of words as 
# features and the sentiment labels as the response variable
#
# This may take a few minutes to run
print ("Training the random forest...")
forest = forest.fit( dataFeaturesTweets3tfidf, dfTweets["airline_sentiment"] )

# random forest performance through cross vaidation 
print (forest)
print (np.mean(cross_val_score(forest, dataFeaturesTweets3tfidf, dfTweets["airline_sentiment"],cv=10)))

Training the random forest...


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.


building tree 1 of 100building tree 2 of 100building tree 3 of 100building tree 4 of 100



building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100building tree 26 of 100

building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100

[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    5.5s



building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100
building tree 45 of 100
building tree 46 of 100
building tree 47 of 100
building tree 48 of 100
building tree 49 of 100
building tree 50 of 100
building tree 51 of 100
building tree 52 of 100
building tree 53 of 100
building tree 54 of 100
building tree 55 of 100
building tree 56 of 100
building tree 57 of 100
building tree 58 of 100
building tree 59 of 100
building tree 60 of 100
building tree 61 of 100
building tree 62 of 100
building tree 63 of 100
building tree 64 of 100
building tree 65 of 100
building tree 66 of 100
building tree 67 of 100
building tree 68 of 100
building tree 69 of 100
building tree 70 of 100
building tree 71 of 100
building tree 72 of 100
building tree 73 of 100
building tree 74 of 100
building tree 75 of 100
building tree 76 of 100
building tree 77 of 100
building tree 78 of 100
building tree 7

[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   16.4s finished


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=False, random_state=None, verbose=2,
                       warm_start=False)


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    6.3s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   19.7s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    6.2s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   17.2s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    7.1s
[Parallel(n_jobs=-1)]:

0.7207650273224044


[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   16.7s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished


### Summarizing my understanding and performance of my model in this dataset (8 marks)

Text preprocessing should be the first step in NLP. The importance of preprocessing increases in NLP due to noise or unclear data extracted or collected from different sources. Feeding raw data to build models without preprocessing is not recommended because the preprocessing of text directly improves the model's performance.

Text preprocessing varies from problem to problem. This means we cannot apply the same text preprocessing techniques used for one NLP problem to another NLP problem. For example, in sentiment analysis classification problems, we can remove or ignore numbers within the text because numbers are not significant, however, ignoring the numbers if the text is related to a financial statement, might not be advisable as numbers might play a key role.

CountVectorizer counts the word frequencies, as simple as that, while as TFIDFVectorizer the value increases proportionally to count, but is inversely proportional to frequency of the word in the corpus (IDF). The inverse document frequency adjusts for the fact that some words appear more frequently in general. This also makes TFIDF to be sensitive to symmetries in the corpus distribution.

Model performace with all three methods used are very similar in the range between 0.7237 (CountVectorizer), 0.7242 (TfidfTransformer) and 0.7207 TfidfVectorizer

### (Thinking on validating train - test.  The following cells are only for personal reference)

    X = dfBankDataDm2.drop("Target_yes" , axis=1)
    y = dfBankDataDm2.pop("Target_yes")

    # nötig
    # Specify the data 
    X=dfTweets.iloc[:,[0,1,2,3,4,5,6,7,9,10,11,12,13]]

    # Specify the target labels and flatten array
    y= dfTweets.iloc[:,8]

    # Split the data up in train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

    # y_train =  np.array(y_train)
    # y_test =  np.array(y_test)